In [1]:
from gymnasium.wrappers import RecordVideo

In [2]:
import import_ipynb
import random, datetime
from pathlib import Path

import gymnasium
from gymnasium.wrappers import FrameStack, GrayScaleObservation, TransformObservation
import gymnasium as gym

In [3]:
from trained_agent import SpaceInvader

importing Jupyter notebook from trained_agent.ipynb
importing Jupyter notebook from neural.ipynb


In [4]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, truncated, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, truncated, info

In [5]:
import random, datetime
from pathlib import Path

import gymnasium
from gymnasium.wrappers import FrameStack, GrayScaleObservation, TransformObservation


from metrics import MetricLogger
#from agent import SpaceInvader
from wrappers import ResizeObservation

env = gymnasium.make("ALE/SpaceInvaders-v5", mode=2,render_mode="rgb_array")

#video recording
output_dir = "video"
video_output_frequency = 1
env = RecordVideo(env, output_dir, episode_trigger=lambda episode_id: episode_id == 1 or (episode_id != 0 and (episode_id) % video_output_frequency == 0))
env.metadata['render_fps'] = 24


env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env, keep_dim=False)
env = ResizeObservation(env, shape=84)
env = TransformObservation(env, f=lambda x: x / 255.)
env = FrameStack(env, num_stack=4)

env.reset()

save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

checkpoint = Path('checkpoints/2023-05-11T19-11-46/SpaceInvader_net_13.chkpt')
spaceInvader = SpaceInvader(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir, checkpoint=checkpoint)
spaceInvader.exploration_rate = spaceInvader.exploration_rate_min

logger = MetricLogger(save_dir)
episodes = 100
for e in range(episodes):

    state = env.reset()
    state = state[0]
    # Play the game!
    while True:

        # 3. Show environment (the visual) [WIP]
        # env.render()

        # 4. Run agent on the state
        action = spaceInvader.act(state)
        # 5. Agent performs action
        result = env.step(action)
        next_state = result[0]
        reward = result[1]
        done = result[2]
        info = result[3]
        
        # 6. Remember
        spaceInvader.cache(state, next_state, action, reward, done)

        # 7. Learn
        q, loss = spaceInvader.learn()

        # 8. Logging
        logger.log_step(reward, loss, q)

        # 9. Update state
        state = next_state
        # 10. Check if end of game
        if done or info:
            break
    
    env.close()
    logger.log_episode()

    if e % 1 == 0:
        logger.record(
            episode=e,
            epsilon=spaceInvader.exploration_rate,
            step=spaceInvader.curr_step
        )

importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from wrappers.ipynb


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/Users/tobiaspoulsen/Library/Caches/pypoetry/virtualenvs/tobiaspoulsen--jf9GMjB-py3.11/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:79: UserWarning: WARN: Overwriting existing videos at /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
<string>:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
/Users/tobiaspoulsen/Library/Caches/pypoetry/virtualenvs/tobiaspoulsen--jf9GMjB-py3.11/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bo

Loading model at checkpoints/2023-05-11T19-11-46/SpaceInvader_net_13.chkpt with exploration rate 0.19691163516176424
Moviepy - Building video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-1.mp4
Episode 0 - Step 265 - Epsilon 0.1 - Mean Reward 540.0 - Mean Length 265.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 2.277 - Time 2023-05-21T18:28:18


Moviepy - Building video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-1.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-71.mp4
Episode 71 - Step 17325 - Epsilon 0.1 - Mean Reward 511.458 - Mean Length 240.625 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 1.646 - Time 2023-05-21T18:31:00


Moviepy - Building video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-72.mp4.
Moviepy - Writing video /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-72.mp4



Moviepy - Done !
Moviepy - video ready /Users/tobiaspoulsen/Desktop/spaceIvander2.0/Project_DQN/video/rl-video-episode-72.mp4
Episode 72 - Step 17495 - Epsilon 0.1 - Mean Reward 508.904 - Mean Length 239.658 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 1.699 - Time 2023-05-21T18:31:02


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>